In [1]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf
import pandas as pd

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [2]:
attackdata = attackToExcel.get_stix_data("enterprise-attack")
techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack")
tactics_data = stixToDf.tacticsToDf(attackdata, "enerprise-attack")
groups_data = stixToDf.groupsToDf(attackdata, "enterprise-attack")
software_data = stixToDf.softwareToDf(attackdata, "enterprise-attack")
mitigations_data = stixToDf.mitigationsToDf(attackdata, "enterprise-attack")
relationship_data = stixToDf.relationshipsToDf(attackdata)

parsing all relationships: 100%|██████████| 14069/14069 [00:00<00:00, 100183.80it/s]


In [3]:
techniques_df = techniques_data["techniques"]
tactics_df = tactics_data["tactics"]
groups_df = groups_data["groups"]
software_df = software_data["software"]
mitigations_df = mitigations_data["mitigations"]
relationship_df = relationship_data["relationships"]

In [4]:
relationship_df.columns = [column.replace(" ", "_") for column in relationship_df.columns]
groupsFromRelationships = relationship_df.query("source_type == 'group'")

In [13]:
IDs = ["T1059"]
temp = groupsFromRelationships
for id in IDs:
    groups = temp[temp.target_ID.str.contains('{}'.format(id))]
    grouplist = groups.source_ID.tolist()
    temp = groupsFromRelationships[groupsFromRelationships['source_ID'].isin(grouplist)]

In [8]:
def get_matches(df: pd.DataFrame, IDs: list) -> pd.DataFrame:
    """
    Returns a dataframe of all groups that match the given technique IDs.
    """
    temp = df
    for id in IDs:
        groups = temp[temp.target_ID.str.contains('{}'.format(id))]
        grouplist = groups.source_ID.tolist()
        temp = df[df.source_ID.isin(grouplist)]
    return temp

In [7]:
matches = get_matches(groupsFromRelationships, IDs)
matches.source_ID.unique().tolist()
len(matches.source_ID.unique().tolist())

1